# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
# '/content/drive/MyDrive/Colab Notebooks/{폴더 이름}' 사용
os.chdir('/content/drive/MyDrive/Colab Notebooks/project')

# 라이브러리

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix

# 데이터 로드

In [ ]:
# 데이터 로드
data = pd.read_csv('data/final_data_senti.csv').drop(columns = 'Unnamed: 0')
data

In [ ]:
# 전처리를 위한 데이터 타입 확인
data.info()

In [ ]:
data = data.dropna()

# 전처리

* 기준 파일: final_data_senti.csv

## Genre: 힙합 -> 랩/힙합

In [ ]:
data.loc[:, 'Genre'] = data.Genre.apply(lambda x: '랩/힙합' if x == '힙합' else x)

In [ ]:
# '힙합' 데이터가 없는 것을 확인
data[data.Genre == '힙합']

## RGB1: R, G, B로

In [ ]:
data['R'] = 0
data['G'] = 0
data['B'] = 0

In [ ]:
data[['R', 'G', 'B']] = data['rgb1'].str.replace('(', '').str.replace(')', '').str.split(',', expand = True)

## top20 여부 컬럼 추가

In [ ]:
data['isTop20'] = data.Rank.apply(lambda x: 1 if x <= 20 else 0 )
data

## 사용 feature 정의

* 제외 feature
  * Rank: isTop10로 대체
  * Title
  * Artist
  * Lyrics
  * color1, color2, color3: 'R', 'G', 'B'로 대체
  * neg, neu, pos: compound로 대체

In [ ]:
using_features = ['Year', 'Month', 'Week', 'Genre', 'BPM', 'Duration_sec', 'R', 'G', 'B', 'Featuring', 'compound', 'isTop20']
data = data[using_features]

In [ ]:
data[data.R.isna()]

## 데이터 타입 변경

* encoding: label encoding <br>
장르의 카테고리 수가 많기 때문에 컬럼 수를 늘리지 않는 label encoding이 적합

In [ ]:
# 수치형(number) 변수와 범주형(category) 변수로 구분
num_features = ['Year', 'Month', 'Week', 'BPM', 'Duration_sec', 'R', 'G', 'B', 'compound']
cat_features = ['Genre', 'isTop20']

data[cat_features] = data[cat_features].astype('category')
data[['R', 'G', 'B']] = data[['R', 'G', 'B']].astype('int')

# 모델링

## 데이터 준비

In [ ]:
X = data.drop(columns = 'isTop20')
y = data['isTop20']

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X[num_features])

encoder = LabelEncoder()
X_genre_encoded = encoder.fit_transform(X['Genre']).reshape(-1, 1)
le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(f"feature: Genre, mapping value: {le_name_mapping}")

encoder = LabelEncoder()
X_featuring_encoded = encoder.fit_transform(X['Featuring']).reshape(-1, 1)
le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(f"feature: Featuring, mapping value: {le_name_mapping}")

X_scaled = np.hstack((X_num_scaled, X_genre_encoded, X_featuring_encoded))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)

## 모델 정의

### Logistic Regression

In [ ]:
model = LogisticRegression()

### Naive Bayes

In [ ]:
model = GaussianNB()

### SVM

[커널 종류]
* linear: 기본 커널, 데이터가 선형적으로 구분 가능한 경우, feature가 많아 고차원 연산을 필요로 하는 경우
* RBF: 비선형적 데이터에 적합, 거리 기반 함수, 구분 경계에서 멀리 있을수록 영향력 감소
* polynomial: 선형 분류 확장 버전, 비선형 관계가 다항식으로 표현 가능할 때 사용
* sigmoid: 비선형 데이터에 적용 가능, RBF를 주로 사용함

In [ ]:
model = SVC(kernel = 'linear')

### Decision Tree

In [ ]:
model = DecisionTreeClassifier(random_state = 42)

### Random Forest

In [ ]:
model = RandomForestClassifier(random_state = 42)

## 모델 학습 & 예측

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
unique, counts = np.unique(y_pred, return_counts = True)
uniq_cnt_dict = dict(zip(unique, counts))
print(uniq_cnt_dict)

## 성능 평가

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")
print('Classification Report')
print(classification_report(y_test, y_pred))

print('Confusion matrix')
print(confusion_matrix(y_test, y_pred))